In [1]:
from googlevoice import Voice
from googlevoice import util
import sys
import fileinput
import csv
import pprint
import pandas as pd
import random
import time
import BeautifulSoup

In [3]:
# Use this function to generate text body at time of send. 
# Also saves your ass if you accidentally target the same person multiple times.

message_components = [
    [
        "Hi {}! ", 
        "Hey there {}! ",
        "Hey {}! ",
        "Hi there {}! "
    ],
    [
        "This is {} with the Trapezoid of Discovery. ",
        "I'm {} with the Trapezoid of Discovery. ",
        "My name's {} and I'm working with the Trapezoid of Discovery. ",
        "My name is {}, with the Trapezoid of Discovery. ",
        "I'm {}, with the Trapezoid of Discovery. ",
        "My name is {}. I'm working with the Trapezoid of Discovery. "
    ],
    [
        "I wanted to check and see if you had a plan to vote today, or if you've already voted. Your polling location is {}, open til 8pm. ",
        "Do you have a plan to vote today, or have you already voted? Your polling location is {}. It's be open until 8! ",
        "I wanted to check if you had a plan for voting today. Your voting location is {}. It's open until 8pm. ",
        "Are you going to vote today? Have you already voted? You can vote at {} until 8pm. "
    ],
    [
        "Please let me know if you have any questions, especially about Space Horse. Thank you very much!",
        "If you have any questions, especially questions about Space Horse, I'm happy to help. Thank you very much!",
        "If you need anything, or have any questions (especially about Space Horse), I'm very happy to help. Thank you very much!"]
]

def write_message():
    message = ''
    for message_part in message_components:
        message += random.choice(message_part)
    return message
      
        
write_message()
    

"Hi there {}! I'm {} with the Trapezoid of Discovery. I wanted to check and see if you had a plan to vote today, or if you've already voted. Your polling location is {}, open til 8pm. Please let me know if you have any questions, especially about Space Horse. Thank you very much!"

In [ ]:
# Add the login information for any Google Voice accounts you'd like to use below.
# Obviously plaintext passwords are horribly unsafe so be careful.

texters = [
    {
        "name":"Heath Rubbit",
        "email":"fake.email@gmail.com",
        "pass":"password12345"
    },
    {
        "name":"Camacho",
        "email":"fake.email2@gmail.com",
        "pass":"password12345"
    },
    {
        "name":"Hoffbrow",
        "email":"fake.email3@gmail.com",
        "pass":"password12345"
    }
]

In [ ]:
# generates a dict of texter name and corresponding GV client

clients = {}
for texter in texters:
    username = texter['email'] 
    password = texter['pass']
    
    voice = Voice()
    voice.login(username, password)
    print texter['name']
    
    clients[texter['name']]= voice

clients

In [12]:
# Pull in the list you get from VAN or wherever in CSV form. 
# Here I restricted it to four columns, but you can dick around in Pandas to get what you need

df = pd.read_csv('eday.csv')
df.columns = ['id','lname','fname','number','polling_place']

# Restrict to only people with numbers
df = df[pd.notnull(df['number'])]
df

,id,lname,fname,number,polling_place
0,101,Smith,Beavis,(666) 666-6666,ST LUKE'S CHURCH FOR BIG DUMB BOYS
1,102,Smith,Butt-Head,(420) 555-6969,CLINT'S COMICS


In [11]:
# This code takes a given voter, picks a random texter client, generates a message,
# fills in the personalized info, and sends the message.

counter = 0

for index, row in df.iterrows():
    
    texter_name, texter_client = random.choice(clients.items())
    
    # use below if data returns phone number as float for some reason
    # number = row['number']
    # number = str(int(number))
 
    fname = row['fname']
    lname = row['lname']
    polling_place = row['polling_place'].title()
    recip_id = row['id']
    
    print("{} is texting {} {} ({})").format(texter_name, fname, lname, recip_id)
    
    text = write_message().format(fname, texter_name, polling_place)
    
#     print number
#     print text
#     print ''
#     print ''


    texter_client.send_sms(number,text)
    
    print "Sent!"
    
    counter += 1
    
    if counter % 50 == 0:
        print "\n \n \n {} messages sent \n \n \n ".format(counter)

# adjust the sleep time based on the number of clients you have. I tried to 
# make sure that any given client sent texts no more frequently than once every 
# ten to fifteen seconds and was able to avoid any rate-limiting.
    time.sleep(2)+random.randint(1,3)
    

Buff is texting Beavis Smith (111)
Hi there Beavis! My name is Buff. I'm working with the Trapezoid of Discovery. Are you going to vote today? Have you already voted? You can vote at St Luke'S Church For Big Dumb Boys until 8pm. If you need anything, or have any questions (especially about Space Horse), I'm very happy to help. Thank you very much!
Sent!
Buff is texting  Butt-Head Smith (111)
Hey  Butt-Head! My name's Buff and I'm working with the Trapezoid of Discovery. I wanted to check and see if you had a plan to vote today, or if you've already voted. Your polling location is Clint'S Comics, open til 8pm. If you have any questions, especially questions about Space Horse, I'm happy to help. Thank you very much!
Sent!


In [ ]:
# TEST ZONE

for name, client in clients.iteritems():
    text = "Hi! This is {}".format(name)
    print "Sending from {}".format(name)
    client.send_sms('your_phone_number',text)